In [2]:
#Taller mercado laboral GEIH 2025
#Hecho por: Santiago Niño Duque

In [3]:
#Primero vamos a importar las librerias a usar
import pandas as pd 
import numpy as np

#Importando tambien los datos de la GEIH 2025, directamente desde el repositorio del profesor
df_cgsse = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/cgsse.CSV"
                       ,encoding="latin", sep= ";") 
df_ft = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"
                       ,encoding="latin", sep= ";", low_memory=False) 
df_ocupado = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
                       ,encoding="latin", sep= ";", low_memory=False) 
df_desocupados = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/No%20ocupados.CSV"
                       ,encoding="latin", sep= ";") 

In [4]:
# 1. Calculando la poblacion total de colombia: 
ptc = df_cgsse["FEX_C18"].sum()
print('Población total de colombia:', int(ptc))

Población total de colombia: 52045001


In [ ]:
#Poblacion en edad de trabajar 

sin_edad_trabajar = df_cgsse[df_cgsse['P6040']<12]['FEX_C18'].sum()
edad_trabajar = df_cgsse[df_cgsse['P6040']>=12]['FEX_C18']
total_edad_trabajar = df_cgsse[df_cgsse['P6040']>=12]['FEX_C18'].sum()

print('La poblacion total sin edad para trabajar es:',int(sin_edad_trabajar))
print('La poblacion total con edad para trabjar es:',int(total_edad_trabajar))

df_cgsse['']

La poblacion total sin edad para trabajar es: 8792393
La poblacion total con edad para trabjar es: 43252607


In [ ]:
#Calculado ahora la poblacion economicamente activa
#definimos como poblacion activamente economica como 
claves = ['P6040']
df_poblacionAE = pd.merge(df_ocupado,df_desocupados, on='id', how='inner')

KeyError: 'id'